<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019/blob/master/homeworks/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4

---



## Setup

In [0]:
import sys
from pathlib import Path
from google.colab import drive

# Notebook Library
url = "https://github.com/Data-Science-and-Data-Analytics-Courses/Notebook-Library"
repo = Path("/nblib")
!git clone "{url}" "{repo}"
if repo.parent.as_posix() not in sys.path:
  sys.path.append(repo.parent.as_posix())
%run "{repo}/.Importable.ipynb"

from nblib.imports.Basic import *
from nblib import Git

# Remote
URL = "https://github.com/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019"
REPO = Git.clone(URL, dest="/content")
if REPO.as_posix() not in sys.path:
  sys.path.append(REPO.as_posix())

# Working directory
os.chdir(REPO)

from nblib.imports.DS import *
from setup.Setup import *
from nblib import File

clear_output()

##  Neural Networks
In this problem we will analyze a simple neural network to understand its classification properties. Consider the neural network given in the figure below, with ReLU activation functions (denoted by  f ) on all neurons, and a softmax activation function in the output layer:  
Given an input  x=[x1,x2]T , the hidden units in the network are activated in stages as described by the following equations:  
z1 	=x1W11+x2W21+W01	f(z1)	=max{z1,0}	 	 
 	z2	=x1W12+x2W22+W02	f(z2)	=max{z2,0}	 	 
 	z3	=x1W13+x2W23+W03	f(z3)	=max{z3,0}	 	 
 	z4	=x1W14+x2W24+W04	f(z4)	=max{z4,0}	 
u1 	=f(z1)V11+f(z2)V21+f(z3)V31+f(z4)V41+V01	f(u1)	=max{u1,0}	 	 
 	u2	=f(z1)V12+f(z2)V22+f(z3)V32+f(z4)V42+V02	f(u2)	=max{u2,0}.	 
The final output of the network is obtained by applying the softmax function to the last hidden layer,

$$
o1 \displaystyle = \frac{e^{f(u_1)}}{e^{f(u_1)} +e^{f(u_2)}} \\
o2 \displaystyle = \frac{e^{f(u_1)}}{e^{f(u_1)} +e^{f(u_2)}}
$$o2 	 =ef(u2)ef(u1)+ef(u2). 	 
In this problem, we will consider the following setting of parameters:  
$$
\left[ \begin{array}{ccc} W_{11} &  W_{21} &  W_{01} \\ W_{12} &  W_{22} &  W_{02} \\ W_{13} &  W_{23} &  W_{03} \\ W_{14} &  W_{24} &  W_{04} \\ \end{array}\right] = \left[\begin{array}{ccc} 1 &  0 &  -1 \\ 0 &  1 &  -1 \\ -1 &  0 &  -1 \\ 0 &  -1 &  -1 \end{array}\right],
$$
$$
\left[ \begin{array}{ccccc} V_{11} &  V_{21} &  V_{31} &  V_{41} &  V_{01} \\ V_{12} &  V_{22} &  V_{32} &  V_{42} &  V_{02} \end{array} \right] = \left[ \begin{array}{ccccc} 1 &  1 &  1 &  1 &  0 \\ -1 &  -1 &  -1 &  -1&  2 \end{array}\right].
$$



### Feed Forward Step
Consider the input  x1=3 ,  x2=14 . What is the final output  (o1,o2)  of the network?

Important: Numerical outputs from the softmax function are sometimes extremely close to 0 or 1; if you choose to enter your answers numerically, make sure to report them to at least 9 decimal places! (Alternatively, you may enter your answers symbolically as a function of symbolically  e .)  


In [0]:
w = np.array([[1, 0, -1], 
              [0, 1, -1],
              [-1, 0, -1],
              [0, -1, -1]])
v = np.array([[1, 1, 1, 1, 0],
              [-1, -1, -1, -1, 2]])

x = np.array([[3, 14, 1]])
z =  x.dot(w.T)
fz = np.maximum(z, 0)
fz = np.insert(fz, fz.shape[1], 1, axis=1) # add 1-column to the end
u = np.maximum(fz.dot(v.T), 0)
o = spec.softmax(u, axis=1)
o

array([[9.99999694e-01, 3.05902227e-07]])

## LSTM
The diagram below shows a single LSTM unit that consists of Input, Output, and Forget gates.  
![alt text](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/8e7c0fe7c60323338b556ae490d116cf/asset-v1:MITx+6.86x+1T2019+type@asset+block/images_hw4_p2.png)
The behavior of such a unit as a recurrent neural network is specified by a set of update equations. These equations define how the gates, “memory cell"  ct  and the “visible state"  ht  are updated in response to input  xt  and previous states  ct−1 ,  ht−1 . For the LSTM unit,  

$$
ft 	\displaystyle = \text {sigmoid}(W^{f,h} h_{t-1} + W^{f,x} x_ t + b_ f ) \\
 	it	\displaystyle = \text {sigmoid}(W^{i,h} h_{t-1} + W^{i,x} x_ t + b_ i ) \\
 	ot	\displaystyle = \text {sigmoid}(W^{o,h} h_{t-1} + W^{o,x} x_ t + b_ o ) \\
 	ct	\displaystyle = f_ t \odot c_{t-1} + i_ t \odot \tanh (W^{c,h} h_{t-1} + W^{c,x} x_ t + b_ c) \\
 	ht	\displaystyle = o_ t \odot \tanh (c_ t) \\
$$

where symbol  ⊙  stands for element-wise multiplication. The adjustable parameters in this unit are matrices  Wf,h ,  Wf,x ,  Wi,h ,  Wi,x ,  Wo,h ,  Wo,x ,  Wc,h ,  Wc,x , as well as the offset parameter vectors  bf ,  bi ,  bo , and  bc . By changing these parameters, we change how the unit evolves as a function of inputs  xt .

To keep things simple, in this problem we assume that  xt ,  ct , and  ht  are all scalars. Concretely, suppose that the parameters are given by

 	 Wf,h 	 =0 	 Wf,x 	 =0 	 bf 	 =−100 	 Wc,h 	 =−100 	 	 
 	 Wi,h 	 =0 	 Wi,x 	 =100 	 bi 	 =100 	 Wc,x 	 =50 	 	 
 	 Wo,h 	 =0 	 Wo,x 	 =100 	 bo 	 =0, 	 bc 	 =0 	 	 
We run this unit with initial conditions  h−1=0  and  c−1=0 , and in response to the following input sequence: [0, 0, 1, 1, 1, 0] (For example,  x0=0 ,  x1=0 ,  x2=1 , and so on).  


### LSTM states
Calculate the values  ht  at each time-step and enter them below as an array  [h0,h1,h2,h3,h4,h5] . For ease of calculation, you can assume that you round  ht  to the closest integer in every time-step. E.g., assume  sigmoid(50)≈1  and  tanh(−50)≈−1 .  



In [54]:
W_fh = 0; W_fx = 0; bf = -100         
W_ih = 0; W_ix = 100; bi = 100              
W_oh = 0; W_ox = 100; bo = 0    
W_ch = -100; W_cx = 50; bc = 0

h0 = 0; c0 = 0 # initial states
x = np.array([0, 0, 1, 1, 1, 0])
Wh = np.stack((W_fh, W_ih, W_oh, W_ch), axis=0)
Wx = np.stack((W_fx, W_ix, W_ox, W_cx), axis=0)
b = np.stack((bf, bi, bo, bc), axis=0)

states = np.zeros((len(x)+1, 2))
states[0] = [h0, c0] # current states
for (j,), val in np.ndenumerate(x):
  h, c = states[j] # current states
  zf, zi, zo, zc = h*Wh + val*Wx + b # linear transformations
  f, i, o = map(spec.expit, [zf, zi, zo])

  # New states
  c = f * c + i * np.tanh(zc)
  h = np.rint(o * np.tanh(c))
  states[j+1] = [h, c]
h_updates = states[1:, 0]
print(h_updates)

[ 0.  0.  1. -1.  1. -0.]


In [47]:
spec.expit(100)
np.tanh(1)

0.7615941559557649